In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('/Users/yanglingxi/Documents/GitHub/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')

In [3]:
df.head()

,uuid,region,country,proj_nm,st_date,end_date,awd_amount,awd_value,implementer,sector,...,rfp_rftop,final_res,mind_eval,final_eval,audit,interim_rep,addl_docs,addl_info,corruption,stability
0,1,South and Central Asia (SCA),Afghanistan,Assistance for Afghanistan's Anti-Corruption A...,2010,2013,e) $25 M-$50 M,26642989,Management Systems International (MSI),"Independent Agencies, Civil Society",...,NaN,http://pdf.usaid.gov/pdf_docs/PA00JP3T.pdf,NaN,NaN,NaN,NaN,http://www.msiworldwide.com/project/supporting...,NaN,High Corruption,Unstable
1,2,South and Central Asia (SCA),Afghanistan,Rule of Law Stabilization Program - Formal Com...,2010,2012,e) $25 M-$50 M,33700000,TetraTech ARD,Rule of Law,...,NaN,NaN,NaN,http://pdf.usaid.gov/pdf_docs/PDACU496.pdf,NaN,NaN,NaN,NaN,Very High Corruption,Very Unstable
2,3,South and Central Asia (SCA),Afghanistan,Afghanistan Rule of Law Stabilization Project ...,2011,2012,c) $5 M-$10 M,10000000,"Checchi and Company Consulting, Inc.",Rule of Law,...,NaN,http://pdf.usaid.gov/pdf_docs/PDACY340.pdf,NaN,http://pdf.usaid.gov/pdf_docs/PDACW029.pdf,NaN,NaN,http://www.checchiconsulting.com/index.php?opt...,NaN,Very High Corruption,Very Unstable
3,4,South and Central Asia (SCA),Afghanistan,Political Party Assessment,2012,2012,a) <$1 M,NaN,"Checchi and Company Consulting, Inc.",Elections and Political Processes,...,NaN,http://pdf.usaid.gov/pdf_docs/PNADZ833.pdf,NaN,NaN,NaN,NaN,NaN,NaN,Very High Corruption,Very Unstable
4,30,Europe and Eurasia (E&E),Albania,Justice Sector Strengthening Project (JuST),2010,2015,c) $5 M-$10 M,9012613,Chemonics,"Rule of Law, Civil Society",...,https://www.fbo.gov/index?s=opportunity&mode=f...,NaN,NaN,NaN,NaN,http://pdf.usaid.gov/pdf_docs/PA00HWM1.pdf,NaN,NaN,High Corruption,Unstable


In [4]:
df.drop(['proj_res','rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1,inplace=True)

In [5]:
# Load the regular expression library
import re
# Remove punctuation
df['proj_desc'] = df['proj_desc'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['proj_desc'] = df['proj_desc'].map(lambda x: x.lower())
# Print out the first rows of papers
df['proj_desc'].head()

0    the five components of this program seek to ta...
1    the overall goal of the program was to support...
2    the project focused on strengthening and impro...
3    the political party assessment team aimed to r...
4    the justice sector strengthening project (just...
Name: proj_desc, dtype: object

In [6]:
%%time
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data =df.proj_desc.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long', 'term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic', 'republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking

In [16]:
# Build the bigram and trigram models

trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram

trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
import spacy
import en_core_web_sm
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form trigrams
data_words_trigrams = make_trigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = en_core_web_sm.load()
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['component', 'program', 'seek', 'take', 'already', 'exist', 'improve', 'implement', 'find', 'way', 'ensure', 'progress', 'achieve', 'sustainable', 'project', 'help', 'government', 'islamic_republic', 'strengthen', 'high', 'office', 'oversight', 'hoo', 'make', 'strong', 'effective', 'institution', 'able', 'lead', 'monitor', 'coordinate', 'report', 'effort', 'combat', 'corruption', 'country', 'project', 'work', 'implement', 'program', 'institutional', 'development', 'sustainability', 'also', 'support', 'office', 'carry', 'priority', 'asset', 'registration', 'verification', 'complaint', 'management', 'case', 'track', 'coordination', 'monitor', 'anticorruption', 'performance', 'government', 'agency']]


In [19]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 2), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]]


In [20]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [21]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"improve" + 0.028*"increase" + 0.025*"service" + 0.025*"health" + '
  '0.023*"project" + 0.018*"capacity" + 0.015*"community" + 0.014*"transition" '
  '+ 0.014*"design" + 0.013*"level"'),
 (1,
  '0.036*"project" + 0.018*"support" + 0.018*"capacity" + 0.015*"actor" + '
  '0.015*"local" + 0.014*"provide" + 0.013*"financial" + 0.013*"relate" + '
  '0.013*"program" + 0.013*"critical"'),
 (2,
  '0.032*"program" + 0.019*"aim" + 0.016*"service_delivery" + 0.015*"document" '
  '+ 0.014*"improve" + 0.013*"contribute" + 0.013*"system" + 0.013*"reform" + '
  '0.013*"facilitate" + 0.012*"national"'),
 (3,
  '0.024*"corruption" + 0.022*"program" + 0.019*"support" + 0.018*"government" '
  '+ 0.017*"improve" + 0.013*"trade" + 0.013*"reduce" + 0.012*"investment" + '
  '0.012*"reform" + 0.011*"system"'),
 (4,
  '0.041*"reform" + 0.032*"legal" + 0.029*"project" + 0.023*"health" + '
  '0.021*"policy" + 0.021*"sector" + 0.020*"increase" + 0.019*"support" + '
  '0.018*"palestinian" + 0.016*"i

In [22]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4202927717059278
